In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import re
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [ ]:
names=['URL','Category']
#df=pd.read_csv( "../input/website-classification-using-url/URL Classification.csv")
#df=pd.read_csv('../input/Website classification using URL/URL Classification.csv')
df=pd.read_csv('../input/website-classification-using-url/URL Classification.csv',names=names, na_filter=False)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(df['Category'])
df['Category'] = lb.transform(df['Category'])

data = pd.get_dummies(df,prefix=['Category'], columns = ['Category'])
df = data
df[:2]

In [ ]:
df1 = df[1:4000]
df2 = df[50000:54000]
df3 = df[520000:524000]
df4 =df[535300:539300]
df5 = df[650000:654000]
df6= df[710000:714000]
df7=  df[764200:768200]
df8=  df[793080:797080]
df9=  df[839730:843730]
df10=  df[850000:854000]
df11=  df[955250:959250]
df12=  df[1013000:1017000]
df13=  df[1143000:1147000]
df14=  df[1293000:1297000]
df15=  df[1492000:1496000]
#df6 = df[77000:1562978]
dt=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15], axis=0)
df.drop(df.index[1:4000],inplace= True)
df.drop(df.index[50000:54000],inplace= True)
df.drop(df.index[520000:524000],inplace= True)
df.drop(df.index[535300:539300],inplace= True)
df.drop(df.index[650000:654000],inplace= True)
df.drop(df.index[710000:714000],inplace= True)
df.drop(df.index[764200:768200],inplace= True)
df.drop(df.index[793080:797080],inplace= True)
df.drop(df.index[839730:843730],inplace= True)
df.drop(df.index[850000:854000],inplace= True)
df.drop(df.index[955250:959250],inplace= True)
df.drop(df.index[1013000:1017000],inplace= True)
df.drop(df.index[1143000:1147000],inplace= True)
df.drop(df.index[1293000:1297000],inplace= True)
df.drop(df.index[1492000:1496000],inplace= True)
df.tail()

In [ ]:
X_train=df['URL']
y_train=df.iloc[: , 1:16].values
print(y_train)
y_train.shape

In [ ]:
X_test=dt['URL']
y_test=dt.iloc[: , 1:16].values
print(y_test)
y_test.shape

import re
import nltk
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(2,2))

gs_clf = Pipeline([('vect', stemmed_count_vect),
                   ('tfidf', TfidfTransformer()),
                   ('clf', SGDClassifier(loss='perceptron', penalty='l2',
                    alpha =1e-4 , max_iter=20 ,tol=None)),
   ])
gs_clf = gs_clf.fit(X_train, y_train)


**This is for understandind the basics of stemming it transforms the words to its root**
* >  stem_vectorizer = stemmed_count_vect
* > text1 = 'ï am so much bored because of your meaningless behaviour'
* > print(stem_vectorizer.fit_transform([text1]))
* > print(stem_vectorizer.get_feature_names())

In [ ]:
from keras.preprocessing.text import Tokenizer
def create_and_train_tokenizer(texts):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(texts)
    return tokenizer

from keras.preprocessing.sequence import pad_sequences
def encode_reviews(tokenizer, max_length, docs):
    encoded=tokenizer.texts_to_sequences(docs) 
    padded=pad_sequences(encoded, maxlen=max_length, padding="post")
    return padded

tokenizer=create_and_train_tokenizer(texts = X_train)
vocab_size=len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

max_length=max([len(row.split()) for row in X_train])
print("Maximum length:",max_length)

X_train_encoded = encode_reviews(tokenizer, max_length, X_train)
X_test_encoded = encode_reviews(tokenizer, max_length, X_test)
print('x_train shape:', X_train_encoded.shape)
print('x_test shape:', X_test_encoded.shape)

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras import layers, models
from keras.callbacks import EarlyStopping

In [ ]:
X_train_encoded[:2]

In [ ]:
from keras import layers, models

def create_embedding_model(vocab_size, max_length):
    model=models.Sequential()
    model.add(layers.Embedding(vocab_size, 100, input_length=max_length))

    model.add(layers.Conv1D(1024, 5, activation="relu"))
    #model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D())

    model.add(layers.Conv1D(1024, 5, activation="relu"))
    #model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D())

    model.add(layers.Flatten())
    model.add(layers.Dense(512,  activation="relu")) 
    dropout = Dropout(0.5)
    model.add(layers.Dense(15,  activation="softmax"))   
    return model

embedding_model = create_embedding_model(vocab_size=vocab_size, max_length=max_length)
embedding_model.summary()

from keras.optimizers import SGD
#opt = SGD(lr=0.01, momentum=0.9)
embedding_model.compile(loss='categorical_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])

In [ ]:

#earlyStopping = EarlyStopping(monitor="val_accuracy", patience=1)
modelHistory = embedding_model.fit(X_train_encoded, 
                                   y_train, 
                                   validation_data=(X_test_encoded, y_test),
                                   epochs= 15
                                   )

In [ ]:
print(len(y_train) + len(y_test))
print(len(X_train_encoded))

In [ ]:
_, acc = embedding_model.evaluate(X_train_encoded, y_train, verbose=0)
print("Train accuracy:{:.2f}".format(acc*100))
_,acc= embedding_model.evaluate(X_test_encoded, y_test, verbose=0)
print("Test accuracy:{:.2f}".format(acc*100))

print(gs_clf.predict(['http://www.banglainfotube.com']))
print(gs_clf.predict(['http://www.gamespot.net/']))

In [ ]:
import tensorflow as tf
#saving the model
embedding_model.save('classification_model.h5')
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('classification_model.h5')
# Show the model architecture
new_model.summary()
# Evaluate the model
loss, acc = new_model.evaluate(X_test_encoded, y_test, verbose=0)
print('Restored model, accuracy 1: {:5.2f}%'.format(100*acc))

''''''''''
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')
# Create a new model instance
model = create_model()
# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')
# Evaluate the model
loss,acc = model.evaluate(X_test_encoded, y_test, verbose=0)
print("Restored model, accuracy 2: {:5.2f}%".format(100*acc))''''



In [ ]:
test_maxlength = 0
for text in X_test:
    length = len(text) 
    if test_maxlength < length:
        test_maxlength = length
        ntext = text
        
print('maxlength is {} and text is = {} '.format(test_maxlength , ntext))